# Пример решения

In [1]:
import pandas
import xarray
import requests
import datetime
import numpy as np
from tqdm import tqdm_notebook as tqdm

## NCEP Dataset

Погодные данные из проекта [NCEP Reanalysis 2](https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis2.html) — усреднённые за день температура воздуха, относительная влажность и компоненты ветра. Данные можно получить с 1979 года.

Загрузите наборы данных в каталог `data/ncep/`:
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/air.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/uwnd.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/rhum.2018.nc

In [2]:
ncep_data = []
year = 2018
for var in ('air', 'uwnd', 'rhum'):
    dataset_filename = 'data/ncep/{}.{}.nc'.format(var, year)
    ncep_data.append(xarray.open_dataset(dataset_filename))
ncep_data = xarray.merge(ncep_data)

/usr/local/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'rhum' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,


## Набор признаков на основе данных NCEP

Ищем наиболее близкий к точке узел сетки в наборе NCEP, в качестве признаков значения переменных зарегистрированные в день регистрации точки и агрегированные показатели за период от 1 до 3х недель до момента регистрации точки.

In [3]:
def extract_features(row):
    point = ncep_data.sel(
        lon=row['longitude'],
        lat=row['latitude'],
        level=1000,
        method='nearest',
    )

    p1w = point.rolling(time=7).mean()
    p2w = point.rolling(time=14).mean()
    p3w = point.rolling(time=21).mean()
    
    date = row['date']
    v = point.sel(time=date)
    v1w = p1w.sel(time=date)
    v2w = p2w.sel(time=date)
    v3w = p3w.sel(time=date)
    
    return {
        'fire_id': row['fire_id'],
        'fire_type': row['fire_type'],
        'fire_type_name': row['fire_type_name'],
        'date': row['date'], 
        'temperature': v.air.values.item(0),
        'humidity': v.rhum.values.item(0),
        'uwind': v.uwnd.values.item(0),
        't1w': v1w.air.values.item(0),
        't2w': v2w.air.values.item(0),
        't3w': v3w.air.values.item(0),
        'h1w': v1w.rhum.values.item(0),
        'h2w': v2w.rhum.values.item(0),
        'h3w': v3w.rhum.values.item(0)
    }

## Выборка для обучения

In [4]:
df_train = pandas.read_csv('data/wildfires_train.csv')
df_subsample = df_train.query('(date > "2018") & (date < "2019")').sample(n=2000)

df_features = []
for i, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0]):
    features = extract_features(row)
    df_features.append(features)
df_features = pandas.DataFrame(df_features)
df_features.set_index('fire_id', inplace=True)

## Обучение классификатора

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [6]:
X = df_features.iloc[:, 3:].fillna(0)
y = df_features['fire_type']

In [7]:
fire_classifier = GradientBoostingClassifier()

In [8]:
cross_val_score(
    fire_classifier, 
    X, y, 
    cv=5, 
)

array([0.4009901 , 0.47014925, 0.42892768, 0.48492462, 0.4835443 ])

In [9]:
fire_classifier.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Решение для отправки в систему

In [10]:
import pickle

with open('solution/model.pickle', 'wb') as fout:
    pickle.dump(fire_classifier, fout, protocol=pickle.HIGHEST_PROTOCOL)

## Прогнозирование на новых данных

In [11]:
df_predictions = pandas.DataFrame(
    fire_classifier.predict_proba(X),
    index=df_features.index,
    columns=[
        'fire_{}_prob'.format(class_id)
        for class_id in fire_classifier.classes_
    ],
)
df_predictions.head()

,fire_1_prob,fire_2_prob,fire_3_prob,fire_4_prob,fire_5_prob,fire_6_prob,fire_8_prob,fire_9_prob,fire_10_prob,fire_11_prob
fire_id,,,,,,,,,,
143429,0.001159,0.000017,0.007399,0.004252,0.003552,0.286441,0.078428,0.435200,0.137525,0.046028
138178,0.000705,0.000052,0.058708,0.006433,0.001608,0.161008,0.017639,0.139629,0.589347,0.024871
155954,0.000730,0.000005,0.021415,0.003711,0.002800,0.340530,0.027788,0.415358,0.043275,0.144389
157753,0.000566,0.000016,0.008710,0.410101,0.001003,0.068589,0.013258,0.371023,0.094216,0.032518
144402,0.000595,0.000006,0.004892,0.002338,0.002578,0.269705,0.040485,0.493960,0.045141,0.140300


In [12]:
df_predictions.to_csv('data/sample_predictions.csv', index_label='fire_id')